# Weather Analysis

In [ ]:
import config 
from datetime import timedelta, date
import fetch
import folium
from keplergl import KeplerGl
import pandas as pd

## Finding Relevant Weather Data Points

In [ ]:
# Load all distinct coordinates from original weather data
OG_weather_points = pd.read_csv('./weather/epvars90m_ChiLoopOnly_2015.Loop.csv', usecols=['lat','lon'], nrows=880)

# Initialize map
locations = OG_weather_points[['lat', 'lon']].values.tolist()
OG_weather_map = folium.Map(locations[len(locations)//2 - 1], zoom_start=14)

# Create permimeter around weather points
minLat = OG_weather_points.lat.min()
maxLat = OG_weather_points.lat.max()
minLon = OG_weather_points.lon.min()
maxLon = OG_weather_points.lon.max()

points = [[minLat, minLon], [minLat, maxLon], [maxLat, minLon], [maxLat, maxLon]]
folium.Rectangle(bounds=points, color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.2).add_to(OG_weather_map)
folium.Rectangle(bounds=points, color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.2).add_to(OG_weather_map)

# Place marker on map for each building 
buildingsDF = pd.read_csv('./buildings/chi0_90m_coord2bldg_smc.csv')

for id, row in buildingsDF.iterrows():
    folium.Marker([row['Lat'],row['Lon']],tooltip=row['BLDGID']).add_to(OG_weather_map)

OG_weather_map

In [ ]:
# Load in all distinct coordinates in reduced weather dataset
reduced_weather_points = pd.read_csv('./weather/weather.csv', usecols=['lat','lon'], nrows=390)

# Initialize map
reduced_weather_map = folium.Map(locations[len(locations)//2 - 1], zoom_start=14)

# Create permimeter around weather points
minLat = reduced_weather_points.lat.min()
maxLat = reduced_weather_points.lat.max()
minLon = reduced_weather_points.lon.min()
maxLon = reduced_weather_points.lon.max()

points = [[minLat, minLon], [minLat, maxLon], [maxLat, minLon], [maxLat, maxLon]]
folium.Rectangle(bounds=points, color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.2).add_to(reduced_weather_map)
folium.Rectangle(bounds=points, color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.2).add_to(reduced_weather_map)

# Place marker on map for each building
for id, row in buildingsDF.iterrows():
    folium.Marker([row['Lat'],row['Lon']],tooltip=row['BLDGID']).add_to(reduced_weather_map)

reduced_weather_map

## Mapping Geographic Variations in Weather Data

In [ ]:
def getDayMap(m,d,var):
    '''Function that returns dataframe of weather data after fetching from SQLite database '''
    if var[-2] == '-':
        var = f'`{var}`'

    df = fetch.get_weather(usecols=f'time,lat,lon,{var}', datetime=date(2015,m,d))
    df['time'] = df['time'].apply(lambda x: pd.Timestamp(x[:10] + ' ' + x[11:]))

    if var == 'temp_K':
        df[var] = df[var].apply(lambda x: (x-273)*1.8 +32)
        df.rename(columns={var: 'temp_F'})

    return df

### Displaying Weather Data
Click on drop menu on left side of map to reveal map configurations. Toggle visibility of measurement points to display desired measurement. Click on filter option to choose corresponding filter. Press play to start timed visualization. Toggle legend on/off on right side of map.

In [ ]:
# day with greatest variance in temperature

feb20 = {
    'temp_K': None,
    'WindSpd_ms-1': None,
    'Longwave_Wm-2': None,
    'RH_pct': None
}

for id, key in feb20.items():
    feb20[id] = getDayMap(2,20,id)

KeplerGl(height=600, data=feb20, config=config.feb20)

In [ ]:
# day with greatest variance in wind speed

oct20 = {
    'temp_K': None,
    'WindSpd_ms-1': None,
    'Longwave_Wm-2': None,
    'RH_pct': None
}

for id, key in oct20.items():
    oct20[id] = getDayMap(10,20,id)

KeplerGl(height=600, data=oct20, config=config.oct20)

In [ ]:
# day with greatest variance in longwave radiation

apr10 = {
    'temp_K': None,
    'WindSpd_ms-1': None,
    'Longwave_Wm-2': None,
    'RH_pct': None
}

for id, key in apr10.items():
    apr10[id] = getDayMap(4,10,id)

KeplerGl(height=600, data=apr10, config=config.apr10)

In [ ]:
# day with greatest variance in relative humidity

apr14 = {
    'temp_K': None,
    'WindSpd_ms-1': None,
    'Longwave_Wm-2': None,
    'RH_pct': None
}

for id, key in apr14.items():
    apr14[id] = getDayMap(4,14,id)

KeplerGl(height=600, data=apr14, config=config.apr14)